In [1]:
import sys


sys.path.append("../src/")

In [2]:
from sennet.core.mmap_arrays import create_mmap_array, read_mmap_array

In [3]:
from plyfile import PlyData, PlyElement
import numpy as np

In [4]:
plydata = PlyData.read("/home/clay/research/kaggle/sennet/data_dumps/label3.ply")

In [5]:
plydata.elements[0]

PlyElement('vertex', (PlyProperty('x', 'float'), PlyProperty('y', 'float'), PlyProperty('z', 'float')), count=4109798, comments=[])

In [6]:
points_x = np.ascontiguousarray(plydata.elements[0].data["x"].copy()).astype(int)
points_y = np.ascontiguousarray(plydata.elements[0].data["y"].copy()).astype(int)
points_z = np.ascontiguousarray(plydata.elements[0].data["z"].copy()).astype(int)

points_x, points_y = points_y, points_x

print(f"{points_x.shape = }")
print(f"{points_y.shape = }")
print(f"{points_z.shape = }")

points_x.shape = (4109798,)
points_y.shape = (4109798,)
points_z.shape = (4109798,)


In [7]:
points_x

array([740, 705, 705, ..., 807, 808, 809])

In [8]:
out_data = create_mmap_array(
    "/home/clay/research/kaggle/sennet/data_dumps/processed/kidney_3_pl/cleaned_label", 
    shape=[1035, 1706, 1510],
    dtype=np.uint8
)
print(out_data.shape)

[1035, 1706, 1510]


In [9]:
print(f"{points_x.min()=}, {points_x.max()=}")
print(f"{points_y.min()=}, {points_y.max()=}")
print(f"{points_z.min()=}, {points_z.max()=}")

points_x.min()=345, points_x.max()=1255
points_y.min()=72, points_y.max()=1394
points_z.min()=52, points_z.max()=940


In [10]:
out_data.data[:] = 0
out_data.data[points_z, points_y, points_x] = 1

In [11]:
k3m_mmap = read_mmap_array("/home/clay/research/kaggle/sennet/data_dumps/processed/kidney_3_merged/label", mode="r")

In [12]:
from tqdm.notebook import tqdm


for i in tqdm(range(out_data.shape[0])):
    if 496 <= i <= 996:
        out_data.data[i, ...] = k3m_mmap.data[i, ...]
    else:
        out_data.data[i, ...] = out_data.data[i, ...] | k3m_mmap.data[i, ...]

  0%|          | 0/1035 [00:00<?, ?it/s]

In [13]:
out_data.data.flush()